<a href="https://colab.research.google.com/github/Christian-Oliveira/web-scraping-beautifulsoup/blob/main/web_scraping_alura_motors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install wget

### **Inicializando bibliotecas Acessando o site alvo e covertendo para objeto BeautifulSoup**

In [2]:
import requests
from bs4 import BeautifulSoup

response = requests.get("http://alura-site-scraping.herokuapp.com/index.php").text
soup = BeautifulSoup(response, "html.parser")

### **Função para capturar informações dos cards**

In [4]:
cards = []
card = {}

publication = soup.find('div', {'class': "well card"})
card['value'] = publication.find('p', {'class': "txt-value"}).getText()
informations = publication.find('div', {'class': "body-card"}).findAll('p')

for info in informations:
  key = info.get('class')[0].split('-')[-1]
  card[key] = info.getText()

### **Capturando items dos veiculos nos anuncios**

In [ ]:
items = publication.find('div', {'class': "body-card"}).findAll('li')
items.pop()
props = []

for item in items:
  item_formated = item.getText().replace('► ', '')
  props.append(item_formated)

card['items'] = props
card

### **Criando DataFrame com Pandas**

In [29]:
import pandas as pd

dataset = pd.DataFrame.from_dict(card, orient='index').T
dataset.to_csv('./drive/MyDrive/Colab Notebooks/data/dataset.csv', sep=';', index=False, encoding='utf-8-sig')

### **Capturando fotos dos veículos nos anuncios e fazendo download**

In [ ]:
import wget

img_source = publication.find('div', {'class': "image-card"}).img.get('src')
name_image = img_source.split('/')[-1]
wget.download(img_source, out=f"./drive/MyDrive/Colab Notebooks/img/{name_image}")

### **Acessando proximas páginas dos anuncios**

In [75]:
page = int(soup.find('span', {'class': "info-pages"}).getText().split()[-1])
page

25

# **Web Scrapping do curso da Alura**

In [77]:
import requests, wget
from bs4 import BeautifulSoup
import pandas as pd


URL_BASE = "http://alura-site-scraping.herokuapp.com/index.php"

### Acessando site de anúncios
response = requests.get(URL_BASE).text
soup = BeautifulSoup(response, "html.parser")

total_pages = int(soup.find('span', {'class': "info-pages"}).getText().split()[-1])

for page in range(total_pages):
  ### acessando HTML da pagina
  response = requests.get(f"{URL_BASE}?page={page+1}").text
  soup = BeautifulSoup(response, "html.parser")

  ### capturando anúncios na pagina
  publications = soup.find('div', {'id': "container-cards"}).findAll('div', {'class': "well card"})
  for publication in publications:
    card = {}

    ### informações do card
    card['value'] = publication.find('p', {'class': "txt-value"}).getText()
    informations = publication.find('div', {'class': "body-card"}).findAll('p')
    for info in informations:
      key = info.get('class')[0].split('-')[-1]
      card[key] = info.getText()

    ### capturando informações de acessórios do anuncio
    items = publication.find('div', {'class': "body-card"}).findAll('li')
    items.pop()
    props = []
    for item in items:
      item_formated = item.getText().replace('► ', '')
      props.append(item_formated)

    card['items'] = props

    ### capturando as imagens e fazendo download
    img_source = publication.find('div', {'class': "image-card"}).img.get('src')
    name_image = img_source.split('/')[-1]
    wget.download(img_source, out=f"./drive/MyDrive/Colab Notebooks/img/{name_image}")

    ### add o card a lista de cards
    cards.append(card)

### criando dataframe e gerando csv
dataset = pd.DataFrame(cards)
dataset.to_csv('./drive/MyDrive/Colab Notebooks/data/dataset.csv', sep=';', index=False, encoding='utf-8-sig')

512